# **Importamos librerias**

In [1]:
from etl import UserGenerator
from feature_engineer import FeatureEngineer
from train import Train
from train_with_mlflow import TrainMlflow
from train_with_mlflow_optuna import TrainMlflowOptuna
from mlflow import set_tracking_uri, set_experiment, sklearn, xgboost
import mlflow

# **ETL**

In [2]:
user_generator = UserGenerator(n_samples=25000)

In [3]:
df = user_generator.create_dataset()

Generando datos sintéticos de usuarios para targeting de promociones...
✅ Agregados 1225 valores nulos en 'age_group' (5.0%)
✅ Agregados 757 valores nulos en 'location' (3.0%)
✅ Agregados 478 valores nulos en 'device_type' (2.0%)
✅ Agregados 260 valores nulos en 'subscription_type' (1.0%)
✅ Agregados 2046 valores nulos en 'avg_order_value' (8.0%)
✅ Agregados 3702 valores nulos en 'last_purchase_days' (15.0%)
✅ Agregados 2500 valores nulos en 'time_on_site_minutes' (10.0%)
✅ Agregados 2486 valores nulos en 'pages_per_session' (10.0%)
✅ Agregados 3080 valores nulos en 'cart_abandonment_rate' (12.0%)
✅ Agregados 2022 valores nulos en 'purchase_frequency' (8.0%)


# **Feature Engineering**

In [4]:
feature_engineer = FeatureEngineer(df)

In [5]:
df_engineered = feature_engineer.create_features()

In [6]:
df_engineered

,user_id,age_group,location,device_type,subscription_type,days_since_registration,total_purchases,avg_order_value,last_purchase_days,sessions_last_30_days,time_on_site_minutes,pages_per_session,cart_abandonment_rate,purchase_frequency,dar_promocion,total_purchases_per_day,days_between_first_and_last_purchase,bucket_avg_order_value,PRUEBA
0,USER-000001,18-25,Mendoza,Mobile,NaN,38,6,341.58,NaN,2,NaN,1.6,0.075,4.74,1,0.157895,NaN,high,1
1,USER-000002,26-35,Buenos Aires,Mobile,Basic,246,34,215.56,114.0,18,34.1,17.5,0.607,4.15,0,0.138211,132.0,medium,1
2,USER-000003,36-45,Buenos Aires,Mobile,Enterprise,284,21,60.08,NaN,3,43.7,7.5,NaN,2.22,0,0.073944,NaN,low,1
3,USER-000004,36-45,Cordoba,Tablet,Free,343,35,153.66,160.0,19,106.4,7.9,0.154,3.06,1,0.102041,183.0,low,1
4,USER-000005,18-25,Buenos Aires,Mobile,Free,330,14,434.58,97.0,8,55.0,16.8,0.130,1.27,0,0.042424,233.0,high,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,USER-024996,46-55,Mendoza,Mobile,Free,111,49,369.90,45.0,13,83.9,8.7,0.453,13.24,1,0.441441,66.0,high,1
24996,USER-024997,26-35,Rosario,Mobile,Free,292,30,390.38,NaN,29,93.1,8.7,0.003,NaN,1,0.102740,NaN,high,1
24997,USER-024998,36-45,La Plata,Mobile,Basic,261,41,178.08,162.0,0,53.7,1.1,NaN,4.71,1,0.157088,99.0,medium,1
24998,USER-024999,18-25,Mendoza,Desktop,Premium,4,28,12.07,66.0,15,87.4,4.7,NaN,28.00,0,7.000000,-62.0,low,1


# **Step 1: Modelando sin mlflow**

In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
numeric_features = ['days_since_registration', 'total_purchases', 'avg_order_value', 
                   'last_purchase_days', 'sessions_last_30_days', 'time_on_site_minutes', 
                   'pages_per_session', 'cart_abandonment_rate', 'purchase_frequency']
categorical_features = ['age_group', 'location', 'device_type', 'subscription_type']
target_column = 'dar_promocion'
test_size = 0.25

#model = LogisticRegression(random_state=42, max_iter=1000)

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42, n_estimators=100)


In [9]:
train = Train(df_engineered, numeric_features, categorical_features, target_column, model, test_size)
pipeline = train.train()

In [10]:
pipeline

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


# **Step 2: Modelando con MLFlow**

## **Modelo con Random Forest**

In [14]:
experiment_name = "promociones-random-forest"
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name)

# Enable autologging for sklearn models
mlflow.sklearn.autolog(
    log_input_examples=True,
    log_model_signatures=True,
    log_models=True,
    disable=False,
    exclusive=False,
    disable_for_unsupported_versions=False,
    silent=False,
    max_tuning_runs=5
)

2025/09/16 20:38:35 INFO mlflow.tracking.fluent: Experiment with name 'promociones-random-forest' does not exist. Creating a new experiment.


In [15]:
params_random_forest = {
    'n_estimators': 100,
    'max_depth': 10,
    'min_samples_split': 2,
    'min_samples_leaf': 1
}
model = RandomForestClassifier(random_state=42, **params_random_forest)


In [16]:
train_mlflow = TrainMlflow(df_engineered, numeric_features, categorical_features, 
target_column, model, test_size, params_random_forest, mlflow_setup= mlflow)
train_mlflow.train()

2025/09/16 20:41:37 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


MLflow Run ID: 987953b1e7a84bb5a904a227565d2eeb
Tracking URI: http://127.0.0.1:5000
Train Accuracy: 0.8353
Test Accuracy: 0.4963
🏃 View run bright-shrimp-68 at: http://127.0.0.1:5000/#/experiments/1/runs/987953b1e7a84bb5a904a227565d2eeb
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


(Pipeline(steps=[('preprocessor',
                  ColumnTransformer(transformers=[('num',
                                                   Pipeline(steps=[('imputer',
                                                                    SimpleImputer(strategy='median')),
                                                                   ('scaler',
                                                                    StandardScaler())]),
                                                   ['days_since_registration',
                                                    'total_purchases',
                                                    'avg_order_value',
                                                    'last_purchase_days',
                                                    'sessions_last_30_days',
                                                    'time_on_site_minutes',
                                                    'pages_per_session',
                                      

## **Modelo con XGB**

In [17]:
from xgboost import XGBClassifier


experiment_name = "promociones-xgboost"
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name)

# Enable autologging for sklearn models
mlflow.xgboost.autolog()

2025/09/16 20:46:22 INFO mlflow.tracking.fluent: Experiment with name 'promociones-xgboost' does not exist. Creating a new experiment.


In [18]:
params_xgb = {
    'n_estimators': 100,
    'max_depth': 10,
    'min_samples_split': 2,
    'min_samples_leaf': 1
}
model_xgb = XGBClassifier(random_state=42, **params_xgb)


In [19]:
train_mlflow = TrainMlflow(df_engineered, numeric_features, categorical_features, 
target_column, model_xgb, test_size, params_xgb, mlflow_setup= mlflow)
train_mlflow.train()

2025/09/16 20:47:06 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


MLflow Run ID: 781ea07d23ee40a4900dcf07dcb56254
Tracking URI: http://127.0.0.1:5000
Train Accuracy: 0.9990
Test Accuracy: 0.5003
🏃 View run receptive-sloth-905 at: http://127.0.0.1:5000/#/experiments/2/runs/781ea07d23ee40a4900dcf07dcb56254
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


(Pipeline(steps=[('preprocessor',
                  ColumnTransformer(transformers=[('num',
                                                   Pipeline(steps=[('imputer',
                                                                    SimpleImputer(strategy='median')),
                                                                   ('scaler',
                                                                    StandardScaler())]),
                                                   ['days_since_registration',
                                                    'total_purchases',
                                                    'avg_order_value',
                                                    'last_purchase_days',
                                                    'sessions_last_30_days',
                                                    'time_on_site_minutes',
                                                    'pages_per_session',
                                      

# **Step 3: Modelo con MLFlow + Optuna**

In [12]:
from sklearn.ensemble import RandomForestClassifier

experiment_name = "promociones-optuna"
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name)

# Define RandomForest parameter distributions
param_distributions = {
    'n_estimators': ('int', 50, 200),
    'max_depth': ('int', 5, 30),
    'min_samples_split': ('int', 2, 10),
    'min_samples_leaf': ('int', 1, 5),
    'max_features': ('categorical', ['sqrt', 'log2', None])
}

# Create trainer for RandomForest
trainer = TrainMlflowOptuna(
    df=df,
    numeric_features=numeric_features,
    categorical_features=categorical_features,
    target_column=target_column,
    model_class=RandomForestClassifier,
    test_size=0.3,
    n_trials=30,
    optimization_metric='f1',  # Optimize for F1 score
    param_distributions=param_distributions,
    model_params={'random_state': 42, 'n_jobs': -1},
    mlflow_setup = mlflow
)

# Run optimization
best_pipeline, run_id, study = trainer.train()

2025/09/17 17:54:51 INFO mlflow.tracking.fluent: Experiment with name 'promociones-optuna' does not exist. Creating a new experiment.
[I 2025-09-17 17:54:51,628] A new study created in memory with name: optuna_RandomForestClassifier


Starting Optuna optimization with 30 trials...
Optimizing for: f1
Model type: RandomForestClassifier


[I 2025-09-17 17:54:51,958] Trial 0 finished with value: 0.5066064966844519 and parameters: {'n_estimators': 161, 'max_depth': 13, 'min_samples_split': 7, 'min_samples_leaf': 1, 'max_features': 'log2'}. Best is trial 0 with value: 0.5066064966844519.
2025/09/17 17:54:51 INFO mlflow.tracking.fluent: Experiment with name 'optuna_RandomForestClassifier' does not exist. Creating a new experiment.
[I 2025-09-17 17:54:52,205] Trial 1 finished with value: 0.5078573180922978 and parameters: {'n_estimators': 142, 'max_depth': 6, 'min_samples_split': 7, 'min_samples_leaf': 3, 'max_features': 'log2'}. Best is trial 1 with value: 0.5078573180922978.


🏃 View run 0 at: http://127.0.0.1:5000/#/experiments/4/runs/2661d4e250d24c0dba01ba735baff822
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4
🏃 View run 1 at: http://127.0.0.1:5000/#/experiments/4/runs/2bf145cd11fd47d7b069a6b16af225e8
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4


[I 2025-09-17 17:54:52,486] Trial 2 finished with value: 0.4982655108124015 and parameters: {'n_estimators': 64, 'max_depth': 5, 'min_samples_split': 4, 'min_samples_leaf': 3, 'max_features': None}. Best is trial 1 with value: 0.5078573180922978.


...

🏃 View run 28 at: http://127.0.0.1:5000/#/experiments/4/runs/da1ed12e3cdc4e26878079a4d2dd6334
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4


[I 2025-09-17 17:55:05,254] Trial 29 finished with value: 0.49798886942128 and parameters: {'n_estimators': 153, 'max_depth': 14, 'min_samples_split': 7, 'min_samples_leaf': 5, 'max_features': 'sqrt'}. Best is trial 16 with value: 0.5085362021206056.


🏃 View run 29 at: http://127.0.0.1:5000/#/experiments/4/runs/76c8ac3343b84583bf4270584f5619bb
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4

Optimization complete!
Best f1: 0.5085
Best parameters: {'n_estimators': 174, 'max_depth': 17, 'min_samples_split': 8, 'min_samples_leaf': 3, 'max_features': 'sqrt'}


2025/09/17 17:55:05 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 17:55:07 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



Best Model MLflow Run ID: 90d2c22bda5e434caa2230f75c16388d
Tracking URI: http://127.0.0.1:5000
Train Accuracy: 0.9895
Test Accuracy: 0.5087
🏃 View run best_model_RandomForestClassifier at: http://127.0.0.1:5000/#/experiments/4/runs/90d2c22bda5e434caa2230f75c16388d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4
